# Univariate Gaussian Mixture Example

In [ ]:
import sys
sys.path.append("../")

import numpy as np
import matplotlib.pyplot as plt
import time

import multiprocessing as mp

from ebc.sequential.iterative_with_convexification import SensitivityBasedFW
from ebc.sequential.iterative_no_convexification import SparseVI

from ebc.gaussian import gaussain_univaraite_log_likelihood

from splitting import split_based_on_ML, split_based_on_sensitivities, split_randomly, distribute
from parallelization import parallelize

from sklearn.mixture import GaussianMixture

import warnings
warnings.filterwarnings(action = "ignore", category = FutureWarning)

import pickle

from analyze_distribution import plot_coresets

## Data Generation

In [ ]:
np.random.seed(123)

full_means = [6, 8, 10]
full_sigmas = [0.8, 0.5, 0.2]
probs = [0.2, 0.5, 0.3]

mixture_inds = np.random.choice([0, 1, 2], size = 5000, replace = True, p = probs)
mixture = []

for i in mixture_inds:
    if i == 0:
        mixture.append(np.random.normal(6, 0.8))
    if i == 1:
        mixture.append(np.random.normal(8, 0.5))
    if i == 2:
        mixture.append(np.random.normal(10, 0.2))

mixture = np.array(mixture).reshape(-1, 1)

plt.hist(mixture, bins = 100);

## Log-likelihood Definition

In [ ]:
def log_likelihood(params, X, y, weights):
    '''
    Returns:
    ----------
    log_lik: np.ndarray(shape = X.shape[0])
    '''
    probs = params[:3]
    mu = params[3:6].reshape(-1, 1)
    sigma = params[6:9]

    ll = 0
    for i in range(3):
        ll += probs[i] * gaussain_univaraite_log_likelihood(X, mu[i], sigma[i])
    
    return ll

def summed_log_likelihood(params, X, y, weights):
    return log_likelihood(params, X, y, weights).sum()

def negative_summed_log_likelihood(params, X, y, weights):
    return -summed_log_likelihood(params, X, y, weights)

def grad_log_likelihood(params, X, y, weights):
    return None

def log_posterior(params, X, y, weights):
    return weights.T @ log_likelihood(params, X, y, weights)

In [ ]:
coreset_sizes = np.arange(100, 310, 10)
len(coreset_sizes)

In [ ]:
# https://stats.stackexchange.com/questions/7440/kl-divergence-between-two-univariate-gaussians
# https://math.stackexchange.com/questions/2614267/can-we-solve-kl-divergence-between-gaussian-mixtures-by-thinking-conditional-cas
def mixture_kl_element(mu0, sigma0, mu1, sigma1, w):
    KL = np.log(sigma1 / sigma0) + (sigma0 ** 2 + (mu1 - mu0) ** 2) / (2 * sigma1 ** 2) - 1/2
    return KL * w

## Testing

In [ ]:
x = mixture

# Sequential
fkl_sequential = []
bkl_sequential = []
time_sequential = []

na = {"log_likelihood": log_likelihood,
      "log_likelihood_start_value": np.ones(9),
      "S": int(0.3 * 0.7 * len(x)),
      "log_likelihood_gradient": None,
      "approx": "MCMC",
      "MCMC_subs_size": int(0.7 * len(x)),
      "log_posterior": log_posterior,
      "log_posterior_start_value": np.ones(9)}

for k in range(20):
      try:
            np.random.seed(120 + k)
            fkl_sequential_k = []
            bkl_sequential_k = []
            time_sequential_k = []

            for i in coreset_sizes:
                  print(k, i)
                  start = time.time()
                  sbfw = SensitivityBasedFW(x)
                  w, I = sbfw.run(k = i, likelihood_gram_matrix = None, norm = "2", norm_attributes = na)
                  w = (w - np.min(w)) / (np.max(w) - np.min(w))
                  time_sequential_k.append(time.time() - start)

                  # Calculate posterior approximation
                  gm = GaussianMixture(3, n_init = 40)
                  gm.fit((w * mixture)[w > 0].reshape(-1, 1))
                  means = gm.means_.flatten()
                  sigmas = gm.covariances_.flatten()
                  ps = gm.weights_.flatten()

                  means_inds = np.argsort(means)
                  means = means[means_inds]
                  sigmas = sigmas[means_inds]
                  ps = ps[means_inds]

                  fkl = 0
                  bkl = 0
                  for j in range(3):
                        fkl += mixture_kl_element(full_means[j], full_sigmas[j], means[j], sigmas[j], ps[j])
                        bkl += mixture_kl_element(means[j], sigmas[j], full_means[j], full_sigmas[j], ps[j])

                  fkl_sequential_k.append(fkl)
                  bkl_sequential_k.append(bkl)

            fkl_sequential.append(fkl_sequential_k)
            bkl_sequential.append(bkl_sequential_k)
            time_sequential.append(time_sequential_k)
      except:
            continue

print(f"FKL: {fkl_sequential}")
print(f"BKL: {bkl_sequential}")
print(f"Time: {time_sequential}")

In [ ]:
fkl_sequential = np.array(fkl_sequential)
bkl_sequential = np.array(bkl_sequential)
time_sequential = np.array(time_sequential)

In [ ]:
x = mixture

# Parallel
fkl_parallel = []
bkl_parallel = []
time_parallel = []

na = {"log_likelihood": log_likelihood,
      "log_likelihood_start_value": np.ones(9),
      "S": int(0.3 * 0.1 * len(x)),
      "log_likelihood_gradient": None,
      "approx": "MCMC",
      "MCMC_subs_size": int(0.1 * len(x)),
      "log_posterior": log_posterior,
      "log_posterior_start_value": np.ones(9)}

for k in range(20):
      np.random.seed(120 + k)
      fkl_parallel_k = []
      bkl_parallel_k = []
      time_parallel_k = []

      for i in coreset_sizes:
            print(f"{k}: {i}")
            fkl_parallel_i = []
            bkl_parallel_i = []
            time_parallel_i = []
            for ind, strat in enumerate([split_randomly, split_based_on_ML]):
                  start = time.time()

                  # Step 1: distribute
                  if ind == 0:
                        full_inds = strat(x)
                  elif ind == 1:
                        gm = GaussianMixture(3, n_init = 40)
                        gm.fit(mixture)
                        # Get probability estimates
                        params = np.hstack((gm.weights_.flatten(), gm.means_.flatten(), gm.covariances_.flatten()))
                        log_liks = log_likelihood(params, mixture, None, None)
                        probs = np.abs(log_liks) / np.sum(np.abs(log_liks))
                        probs = probs.flatten()
                        full_inds = distribute(probs)

                  # Step 2: run
                  w, _ = parallelize(alg = SensitivityBasedFW, x = x, k = int(i // mp.cpu_count()), norm = "2", na = na, distributed_indices = full_inds)
                  w = (w - np.min(w)) / (np.max(w) - np.min(w))

                  time_parallel_i.append(time.time() - start)

                  # Calculate posterior approximation
                  gm = GaussianMixture(3, n_init = 40)
                  gm.fit((w * mixture)[w > 0].reshape(-1, 1))
                  means = gm.means_.flatten()
                  sigmas = gm.covariances_.flatten()
                  ps = gm.weights_.flatten()

                  means_inds = np.argsort(means)
                  means = means[means_inds]
                  sigmas = sigmas[means_inds]
                  ps = ps[means_inds]

                  fkl = 0
                  bkl = 0
                  for j in range(3):
                        fkl += mixture_kl_element(full_means[j], full_sigmas[j], means[j], sigmas[j], ps[j])
                        bkl += mixture_kl_element(means[j], sigmas[j], full_means[j], full_sigmas[j], ps[j])

                  fkl_parallel_i.append(fkl)
                  bkl_parallel_i.append(bkl)

            fkl_parallel_k.append(fkl_parallel_i)
            bkl_parallel_k.append(bkl_parallel_i)
            time_parallel_k.append(time_parallel_i)

      fkl_parallel.append(fkl_parallel_k)
      bkl_parallel.append(bkl_parallel_k)
      time_parallel.append(time_parallel_k)

print(f"FKL: {fkl_parallel}")
print(f"BKL: {bkl_parallel}")
print(f"Time: {time_parallel}")

In [ ]:
fkl_parallel = np.array(fkl_parallel)
bkl_parallel = np.array(bkl_parallel)
time_parallel = np.array(time_parallel)

In [ ]:
data = {
    "fkl_sequential": fkl_sequential,
    "bkl_sequential": bkl_sequential,
    "time_sequential": time_sequential,
    "fkl_parallel": fkl_parallel,
    "bkl_parallel": bkl_parallel,
    "time_parallel": time_parallel
}

with open('../data/univariate_gaussian_mixture.pickle', 'wb') as file:
    pickle.dump(data, file, protocol = pickle.HIGHEST_PROTOCOL)

## Plot

In [ ]:
with open("../data/univariate_gaussian_mixture.pickle", "rb") as file:
    data = pickle.load(file)

fkl_sequential = data["fkl_sequential"]
bkl_sequential = data["bkl_sequential"]
time_sequential = data["time_sequential"]
fkl_parallel = data["fkl_parallel"]
bkl_parallel = data["bkl_parallel"]
time_parallel = data["time_parallel"]

In [ ]:
klsym_sequential = (fkl_sequential + bkl_sequential) / 2
klsym_parallel = (fkl_parallel + bkl_parallel) / 2

coreset_sizes = np.arange(100, 310, 10)

In [ ]:
plt.rcParams.update({'font.size': 22})
fig = plt.figure(figsize = (20, 7))

ax11 = fig.add_subplot(221)
ax12 = fig.add_subplot(223)

ax13 = fig.add_subplot(222)
ax14 = fig.add_subplot(224, sharex = ax13)

ax11.plot(coreset_sizes, np.log(np.nanmedian(klsym_sequential, axis = 0)), label = 'Sequential', 
          linestyle = "solid", linewidth = 2, color = 'black')
ax12.plot(coreset_sizes, np.log(np.nanmedian(klsym_parallel, axis = 0)[:, 0]), label = 'Random split',
          linestyle = "dashed", linewidth = 2, color = 'dimgray')
ax12.plot(coreset_sizes, np.log(np.nanmedian(klsym_parallel, axis = 0)[:, 1]), label = 'ML split',
          linestyle = "solid", marker = "o", linewidth = 2, color = 'maroon')

ax11.spines['bottom'].set_visible(False)
ax11.xaxis.tick_top()
ax11.tick_params(labeltop = False)
ax12.spines['top'].set_visible(False)
ax12.ticklabel_format(useOffset=False)

ax13.spines['bottom'].set_visible(False)
ax13.xaxis.tick_top()
ax13.tick_params(labeltop = False)
ax14.spines['top'].set_visible(False)
ax14.ticklabel_format(useOffset=False)

ax12.set_xlabel("Coreset size")
ax14.set_xlabel("Coreset size")

fig.text(0.04, 0.5, 'Log KL', va='center', rotation='vertical')
fig.text(0.494, 0.5, 'Seconds', va='center', rotation='vertical')

d = .015
kwargs = dict(transform=ax11.transAxes, color='k', clip_on=False)
ax11.plot((-d, +d), (-d, +d), **kwargs)
ax11.plot((1 - d, 1 + d), (-d, +d), **kwargs)

kwargs.update(transform=ax12.transAxes)
ax12.plot((-d, +d), (1 - d, 1 + d), **kwargs)
ax12.plot((1 - d, 1 + d), (1 - d, 1 + d), **kwargs)

kwargs = dict(transform=ax13.transAxes, color='k', clip_on=False)
ax13.plot((-d, +d), (-d, +d), **kwargs)
ax13.plot((1 - d, 1 + d), (-d, +d), **kwargs)

kwargs.update(transform=ax14.transAxes)
ax14.plot((-d, +d), (1 - d, 1 + d), **kwargs)
ax14.plot((1 - d, 1 + d), (1 - d, 1 + d), **kwargs)

fig.legend()
fig.suptitle('Univariate Gaussian Mixture')

ax13.plot(coreset_sizes, np.median(time_sequential, axis = 0), label = 'Sequential',
          linestyle = "solid", linewidth = 2, color = 'black')
ax14.plot(coreset_sizes, np.median(time_parallel, axis = 0)[:, 0], label = 'Random split',
          linestyle = "dashed", linewidth = 2, color = 'dimgray')
ax14.plot(coreset_sizes, np.median(time_parallel, axis = 0)[:, 1], label = 'ML split',
          linestyle = "solid", marker = "o", linewidth = 2, color = 'maroon')

ax11.grid()
ax12.grid()
ax13.grid()
ax14.grid()

plt.savefig("../plots/univariate_gaussian_mixture.eps")

plt.show()